## Dataset preaparation

#### Import necessary packages

In [2]:
import numpy as np
import pandas as pd


#### Import Data form csv

In [5]:
# Define the paths to the files to import
text='/home/antonio/code/pollypineapple1/FED-Predictor/raw_data/Fed_Scrape-2015-2023.csv'
rates='/home/antonio/code/pollypineapple1/FED-Predictor/raw_data/rates.csv'

In [9]:
# Import files
text_df=pd.read_csv(text)
rates_df=pd.read_csv(rates)

#### Check the data

In [7]:
text_df.head()

,Unnamed: 0,Date,Type,Text
0,0,20230412,0,The Federal Reserve on Wednesday released the ...
1,1,20230412,0,The minutes for each regularly scheduled meeti...
2,2,20230412,0,The minutes can be viewed on the Board's website.
3,3,20230412,0,"For media inquiries, e-mail [email protected] ..."
4,4,20230412,0,Minutes of the Federal Open Market Committee\r...


In [8]:
rates_df.head()

,Year,Month,Day,Federal Funds Target Rate,Federal Funds Upper Target,Federal Funds Lower Target,Effective Federal Funds Rate,Real GDP (Percent Change),Unemployment Rate,Inflation Rate
0,1954,7,1,NaN,NaN,NaN,0.80,4.6,5.8,NaN
1,1954,8,1,NaN,NaN,NaN,1.22,NaN,6.0,NaN
2,1954,9,1,NaN,NaN,NaN,1.06,NaN,6.1,NaN
3,1954,10,1,NaN,NaN,NaN,0.85,8.0,5.7,NaN
4,1954,11,1,NaN,NaN,NaN,0.83,NaN,5.3,NaN


In [12]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9974 non-null   int64 
 1   Date        9974 non-null   int64 
 2   Type        9974 non-null   int64 
 3   Text        9974 non-null   object
dtypes: int64(3), object(1)
memory usage: 311.8+ KB


In [13]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Year                          904 non-null    int64  
 1   Month                         904 non-null    int64  
 2   Day                           904 non-null    int64  
 3   Federal Funds Target Rate     462 non-null    float64
 4   Federal Funds Upper Target    103 non-null    float64
 5   Federal Funds Lower Target    103 non-null    float64
 6   Effective Federal Funds Rate  752 non-null    float64
 7   Real GDP (Percent Change)     250 non-null    float64
 8   Unemployment Rate             752 non-null    float64
 9   Inflation Rate                710 non-null    float64
dtypes: float64(7), int64(3)
memory usage: 70.8 KB


#### Remove useless columns and format relevant ones

In [15]:
columns_to_keep = ['Year', 'Month', 'Day', 'Effective Federal Funds Rate']
rates_df=rates_df[columns_to_keep]
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Year                          904 non-null    int64  
 1   Month                         904 non-null    int64  
 2   Day                           904 non-null    int64  
 3   Effective Federal Funds Rate  752 non-null    float64
dtypes: float64(1), int64(3)
memory usage: 28.4 KB


Conversion to datetime

In [25]:
rates_df.loc[:, 'date'] = pd.to_datetime(rates_df[['Year', 'Month', 'Day']])

In [26]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Year                          904 non-null    int64         
 1   Month                         904 non-null    int64         
 2   Day                           904 non-null    int64         
 3   Effective Federal Funds Rate  752 non-null    float64       
 4   date                          904 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 35.4 KB


In [37]:
rates_df.drop(['Year','Month','Day'], axis=1, inplace=True)

KeyError: "['Year', 'Month', 'Day'] not found in axis"

In [38]:
rates_df=rates_df.rename(columns={'Effective Federal Funds Rate': 'rate'})

In [39]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   rate    752 non-null    float64       
 1   date    904 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 14.2 KB


In [40]:
rates_df.head()

,rate,date
0,0.80,1954-07-01
1,1.22,1954-08-01
2,1.06,1954-09-01
3,0.85,1954-10-01
4,0.83,1954-11-01


In [31]:
text_df['Date']=pd.to_datetime(text_df['Date'])

In [33]:
text_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [34]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    9974 non-null   datetime64[ns]
 1   Type    9974 non-null   int64         
 2   Text    9974 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 233.9+ KB


#### Merge the two dataframes based on Date

Drop NAs

In [41]:
rates_df=rates_df.dropna(subset=['rate'])

In [42]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 752 entries, 0 to 901
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   rate    752 non-null    float64       
 1   date    752 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 17.6 KB


In [43]:
sorted_rates=rates_df.sort_values(by='date')
sprted_text=text_df.sort_values(by='Date')